# The Task
We want to create a protocol where a prover can prove that she knows a pythagorean triangle, such that $x^2+y^2=z^2$ without disclosing the actual values

# The plan
As you might've guessed we're going to transform our statement into some cryptographic thing, that is verifiable(complete) and hard to forge (sound).
So now we need to get a set of tools - everything that we need in order to be able to make his computation verifiable. Here's a small breakdown:

1. Setup everything
2. Arithmetization. We will transform our program into polynomials
3. Committing to polynomials
4. Verification

## Setup

Cryptographic protocols are always based on some assumptions about underlying primitives. Plonk is based on polynomial commitment that use elliptic curve pairings. That's why we would first need a prime field, an elliptic curve with extension and some generators  to create Structured Reference String ( common parameters).

SRS is some kind of secret question or query about the book that can show if the prover has read it or not, for example "which is the letter on page 131 and position 19?". Of course with books the problem is that the verifier would either read the book and check it directly or have a limited set of predefined queries, that could be easily fooled 

In [202]:
F = GF(101) # prime field modulo 101
curve = EllipticCurve(F, [0,3]) # y^2 = x^3 + 3
curve

Elliptic Curve defined by y^2 = x^3 + 3 over Finite Field of size 101

Elliptic curve gives us a cyclic group, which means that every point can be generated from a single one:

In [203]:
curve.gens()

((76 : 29 : 1),)

The order of generator ( number of different points it generates) is the same as number of x coordinates and it additionally includs point in infinity 

In [204]:
curve.gens()[0].order()

102


Since the order of the generator is not prime there are other subgroups with a generator that is a multiple $k*G$( or power $G^k$ depending on notation) of our original generator $G$. Then the order of those subgroups is $\frac m {gcd(k,m)}$  We're interested in having a group with an order of 17, so our generator would be $G*6$

In [205]:
for i in range(0,17):
    print(i*6*curve.gens()[0])

(0 : 1 : 0)
(32 : 42 : 1)
(12 : 69 : 1)
(1 : 2 : 1)
(91 : 35 : 1)
(65 : 3 : 1)
(68 : 74 : 1)
(18 : 49 : 1)
(26 : 56 : 1)
(26 : 45 : 1)
(18 : 52 : 1)
(68 : 27 : 1)
(65 : 98 : 1)
(91 : 66 : 1)
(1 : 99 : 1)
(12 : 32 : 1)
(32 : 59 : 1)


We can start from any point we want. This one looks slightly simpler

In [206]:
G1 = curve([1,2]) # point on curve with x=1 and y=2
print('projective coordinates:',G1)
print('affine coordinates:',G1.xy())
print('order of G:', G1.order())

projective coordinates: (1 : 2 : 1)
affine coordinates: (1, 2)
order of G: 17


In [208]:
# multiplesOfG = [G1*i for i in range(17)]
for i in range(0,G1.order()+1):
    multiple = G1*i
    if i%G1.order()!=0:
        print(i,multiple,multiple.xy())
    else:
        print(i,multiple)

0 (0 : 1 : 0)
1 (1 : 2 : 1) (1, 2)
2 (68 : 74 : 1) (68, 74)
3 (26 : 45 : 1) (26, 45)
4 (65 : 98 : 1) (65, 98)
5 (12 : 32 : 1) (12, 32)
6 (32 : 42 : 1) (32, 42)
7 (91 : 35 : 1) (91, 35)
8 (18 : 49 : 1) (18, 49)
9 (18 : 52 : 1) (18, 52)
10 (91 : 66 : 1) (91, 66)
11 (32 : 59 : 1) (32, 59)
12 (12 : 69 : 1) (12, 69)
13 (65 : 3 : 1) (65, 3)
14 (26 : 56 : 1) (26, 56)
15 (68 : 27 : 1) (68, 27)
16 (1 : 99 : 1) (1, 99)
17 (0 : 1 : 0)


In [209]:
G1*0 , G1*17

((0 : 1 : 0), (0 : 1 : 0))

In [ ]:
# cosets
# G102 = curve.gens()[0]
# for i in range(0,17):
#     for j in range(1,7):
#         point = i*6*G102+ j*G102
#         print(i,'\t',j,'\t',i*6+j,'\t',point,'\t', point.order())

## Doubling a point on a curve
If we have a point $P(x,y)$ and $D=P+P$ 
$$D_x = m^2-2*x$$
$$D_y = m*((3*x-m^2) - y)$$
where $m=3*x^2/y$

In [249]:
def double(point):
    m = 3*point.xy()[0]^2/(2*point.xy()[1])
    return curve([m^2 - 2*point.xy()[0], m *(3*point.xy()[0] - m^2) -point.xy()[1]])
print(G, double(G), double(double(G)))
print(G, multiplesOfG[2],multiplesOfG[4])

(1 : 2 : 1) (68 : 74 : 1) (65 : 98 : 1)
(1 : 2 : 1) (68 : 74 : 1) (65 : 98 : 1)


In $\mathbb{R}$ we have infinite points that satisfy the equation, but in  $\mathbb{Z}_p$ for every prime order $r$ ( 17 in our case) that divides $p-1$ we have $r$ generators $G*r + i*p/r*G$ 

We would need pairings for KZG polynomial commitments to work, which is why we need a second group from so called field extension. Field extension is created by adjoining a solution to a irreducible polynomial to the original field. There are points on the curve that are not in the original field just because some of $y$ values are not quadratic residues. To find a field extension for base field modulo $p$ that contains all of the points of order $r=17$ we need to choose a smalles $k$ such that $r|p^k-1$ , we choose 2 because 17 divides 101^2

It works somewhat like complex numbers. First we choose a polynomial $x^2 + 2$ 

In [333]:
polynomials = F['x']
p = polynomials([2,0,1])
p, p.is_irreducible()

(x^2 + 2, True)

Then we define our new field $\mathbb{F}_{p^2}$ . Here besides regular numbers we have a special element $u$ which is the solution for $x^2+2=0$

In [212]:
F2.<u> = F.extension(F['x']([2,0,1]))
curve2 = EllipticCurve(F2, [0,3])
[(g.xy(),g.order()) for g in curve2.gens()]

[((68*u + 85, 65*u + 86), 102), ((45*u + 77, 18*u + 77), 102)]

Our new field order is $101^2$

In [213]:
101*101, F2.order()

(10201, 10201)

This is our new generator G2 in F2 with the same order 17. The first one is suggested by Sage, the second one is it's multiple which is somewhat simpler to work with

In [214]:
G2 = curve2([36,31*u])
print((curve2.gens()[0]*6),(curve2.gens()[0]*6).order())
print(G2, G2.order())

(u + 88 : 56 : 1) 17
(36 : 31*u : 1) 17


We can still check that the generator is indeed on the curve

In [215]:
x_coord = G2.xy()[0]
y_coord = G2.xy()[1]
y_coord^2 == x_coord^3 +3

True

### Finaly generating SRS
Now we're ready to create SRS, that will be used for polynomial commitments later. 

Plonk SRS depends on number of gates and contains powers of a secret $s$ multiplied by generators in both group:  $$SRS = \{s^i * G_1\}, {G_2, s*G_2}$$  where $0\leq i \leq n+5$ and $n$ is the number of gates
For simplicity let $s=2$, of course normally it is generated in a secure MPC protocol and the secret ( aka toxic waste) is discarded afterwards.

Apart from Groth16 the result of Plonk setup can be reused for different circuits, so it's not needed to perform a dedicated ceremony for each circuit. Furthermore the setup is updateable, which means that you can theoretically add some more entropy on top of it and that's it. Since this kind of MPC is relying on a single honest party having this properties gives much more agility and reliability, because you don't have to blindly trust the setup that was performed long ago, and you woudn't have to redo it every time some miniscule changes happen


In [216]:
s=2
SRS = {
    'G1_Points':[s^i*G1 for i in range(9)],
    'G2_Points':[G2, s*G2 ]
}
SRS

{'G1_Points': [(1 : 2 : 1),
  (68 : 74 : 1),
  (65 : 98 : 1),
  (18 : 49 : 1),
  (1 : 99 : 1),
  (68 : 27 : 1),
  (65 : 3 : 1),
  (18 : 52 : 1),
  (1 : 2 : 1)],
 'G2_Points': [(36 : 31*u : 1), (90 : 82*u : 1)]}

# Arithmetization
We have the following equations that encode Pythagorean triangle:

$$ x_1 * x_1 = x_2  $$
$$ x_3 * x_3 = x_4  $$
$$ x_5 * x_5 = x_6  $$
$$ x_2 + x_4 = x_6  $$

It can be rewritten in a new notation where the left operand is always $a$ and right operand is always $b$ and the output of the operation is $c$:
$$ a_1 * b_1 = c_1  $$
$$ a_2 * b_2 = c_2  $$
$$ a_3 * b_3 = c_3  $$
$$ a_4 + b_4 = c_4  $$
That change has actually brought more variables and have broken some of the constraints as all the rows are independent. This will be handled later by introducing "copy constraints"

We encode all of the equations above using a general form of polynomial:


$$q_L*a + q_R*b + q_O*c + q_M*(a*b) $$

Here $q_L, q_R, q_O, q_M, q_A$ correspond to the variables and operations used in a constraint, so that for $ a_1 * b_1 = c_1$  the corresponding values are $q_L=0, q_R=0, q_O=-1, q_M =1$


$$ 0*a_1 + 0*b_1 + 1* a_1 * b_1 +0*q_{const} - 1*c_1 = 0  $$
$$ 0*a_2 + 0*b_2 + 1* a_2 * b_2 +0*q_{const}- 1*c_2 = 0  $$
$$ 0*a_3 + 0*b_3 + 1* a_3 * b_3 +0*q_{const}- 1*c_3 = 0  $$
$$ 1*a_4 + 1*b_4 + 0* a_2 * b_2 +0*q_{const}- 1*c_4 = 0  $$

Now the prover has a valid assignment ( called witness) for this:

$$\vec{q_L} = [0,0,0,1]$$
$$\vec{q_R} = [0,0,0,1]$$
$$\vec{q_M} = [1,1,1,0]$$
$$\vec{q_O} = [-1,-1,-1,-1]$$
$$\vec{q_{const}}=[0,0,0,0]$$
$$\vec{a} = [3,4,5,9]$$
$$\vec{b} = [3,4,5,16]$$
$$\vec{c} = [9,16,25,25]$$


In [292]:
q_L = [0,0,0,1]
q_R = [0,0,0,1]
q_O = [-1,-1,-1,-1]
q_M = [1,1,1,0]
q_const=[0,0,0,0]
a = [3,4,5,9]
b = [3,4,5,16]
c = [9,16,25,25]

Now we have to encode this values as polyonmials using Lagrange Interpolation. We can interpret each element of the vector as evaluation of the polynomial at some point. It would be more intuitive to use natural values like 1,2,3 and so on, but that makes certain computation much harder, and instead we will be using so called Roots of Unity.
The main property of a primitive root of unity often marked as $\omega$ or $\zeta$ is that $\omega^n=1$ and $\omega^i\neq 1, i<n$

It's important to note that we're dealing here with scalar field i.e. a field of scalar multipliers of our generators. It's different from base prime field. You can think about prime field as a square which limits all of the point cooordinates, so it limits moving along x and y axis. So scalar field limits how we move along the curve itself when we add the generator onto itself.

Since we have just 4 gates, we need a 4th root of unity. Since we have a small group it would be easier to find $\omega$ just by trying every element.

The main idea is that for every element $x^{p-1}=1$. So we transform it to $(x^{(p-1)/n})^n=1$ so here $x^{(p-1)/n}$ is our $\omega$


In [321]:
FF=GF(17)
n=4
def primitive_root_of_unity(n):
    q = FF.order() - 1
    if q % n != 0:
        return 1
    while True:
        x = FF.random_element(1, q)
        g = x**(q/FF(n))
        if g**(FF(n)/FF(2)) != 1 and g != 0:
            return g

omega = primitive_root_of_unity(n)
print(omega)
FF(omega)^n

4


1

We have 4 and 13 as 4th roots of unity, we'll take 4 as the lesser one. Now we have to generate so called domain which is used to evaluate (or interpolate) our polynomials. The domain is just a set of powers of $\omega$ that correspond to gates

In [26]:
domain = [omega^i for i in range(0,4)]
print(domain)
k1=2
k2=3
domain = [domain, [k1*omega for omega in domain],[k2*omega for omega in domain]]
domain

[1, 4, 16, 13]


[[1, 4, 16, 13], [2, 8, 15, 9], [3, 12, 14, 5]]

But we need more values for our constraints, so we generate a coset by shifting domain by two different constants $k_1$ and $k_2$. The resulting cosets should not overlap

Next we interpolate the polynomials using our domain

In [58]:
R=FF['x']
f_a = R.lagrange_polynomial( [i for i in zip(domain[0],a)])
f_b = R.lagrange_polynomial( [i for i in zip(domain[0],b)])
f_c = R.lagrange_polynomial( [i for i in zip(domain[0],c)])
f_q_L =  R.lagrange_polynomial( [i for i in zip(domain[0],q_L)])
f_q_R =  R.lagrange_polynomial( [i for i in zip(domain[0],q_R)])
f_q_O =  R.lagrange_polynomial( [i for i in zip(domain[0],q_O)])
f_q_M =  R.lagrange_polynomial( [i for i in zip(domain[0],q_M)])
f_q_const =  R.lagrange_polynomial( [i for i in zip(domain[0],q_const)])

print('f_a',f_a)
print('f_b',f_b)
print('f_c',f_c)
print('f_q_L',f_q_L)
print('f_q_R',f_q_R)
print('f_q_O:',f_q_O)
print('f_q_M:',f_q_M)
print('f_q_const:',f_q_const)

f_a 3*x^3 + 3*x^2 + 13*x + 1
f_b 13*x^3 + 14*x^2 + 3*x + 7
f_c 4*x^3 + 11*x^2 + 5*x + 6
f_q_L 16*x^3 + 4*x^2 + x + 13
f_q_R 16*x^3 + 4*x^2 + x + 13
f_q_O: 16
f_q_M: x^3 + 13*x^2 + 16*x + 5
f_q_const: 0


In [324]:
w = omega
sigma_1 = [
    k1*w^0,#first element of sigma_1 corresponds to a1. it is equal to b1 which has label k1*w^0
    k1*w^1,#a2 = b2 , b elements have k shift
    k1*w^2,#a3 = b3
    k2*w^0 #a4 = c1. c labels have k2 shift
          ]
sigma_2 = [
    w^0, #b1 = a1
    w^1, #b2 = a2
    w^2, #b3 = a3
    k2*w^1 #b4 = c2 
]
sigma_3 = [
    w^3, #c1 = a4
    k1*w^3, #c2 = b4
    k2*w^3, #c3 = c4
    k2*w^2  #c4 = c3
]
sigma_1, sigma_2, sigma_3

([2, 8, 15, 3], [1, 4, 16, 12], [13, 9, 5, 14])

In [329]:
s_sigma_1 = R.lagrange_polynomial(zip(domain[0],sigma_1))
s_sigma_2 = R.lagrange_polynomial(zip(domain[0],sigma_2))
s_sigma_3 = R.lagrange_polynomial(zip(domain[0],sigma_3))
s_sigma_1,s_sigma_2,s_sigma_3

(6*x^3 + 10*x^2 + 13*x + 7, x^3 + 13*x^2 + 4, 14*x^3 + 3*x^2 + 7*x + 6)

### Round 1

We generate 6 random values from $\mathbb{F}_{17}$
The prover has to form following polynomials

$$a(x) = (b_1*x + b_2)*Z_H +f_a(x)$$
$$b(x) = (b_3*x + b_4)*Z_H +f_b(x)$$
$$c(x) = (b_5*x + b_6)*Z_H +f_c(x)$$

where $Z_H$ is a vanishing polynomials, which means that it's equal to zero on all of the domain points. It's very easy to find, since we already know the 4th root of unity $\omega$, so a polynomial $x^4-1$ will always be equal to zero on all of the domain points, since all of them are powers of $\omega$:
$$Z_H(w^i) = ((\omega^i)^4-1) = ((\omega^4)^i-1) = (1^4-1) = 0$$


In [252]:
# b = [FF.random_element() for i in range(6)]
bs = [FF(i) for i in [7,4,11,12,16,2]]
Z_H = R(x^4-1)
bs, Z_H


([7, 4, 11, 12, 16, 2], x^4 + 16)

In [378]:
p_a = R([bs[1],bs[0]])*Z_H + f_a
p_b = R([bs[3],bs[2]])*Z_H + f_b
p_c = R([bs[5],bs[4]])*Z_H + f_c
p_a,p_b,p_c

(7*x^5 + 4*x^4 + 3*x^3 + 3*x^2 + 6*x + 14,
 11*x^5 + 12*x^4 + 13*x^3 + 14*x^2 + 9*x + 12,
 16*x^5 + 2*x^4 + 4*x^3 + 11*x^2 + 6*x + 4)

Now have to commit to evaluations of this polynomials using SRS
$$G*a(s)= G*\sum_i^n a_i*s^i = \sum_i^n a_i*(G*s^i) = \sum_i^n (a_i*SRS_i)$$

In [379]:
a_s = sum([t[0]*t[1] for t in zip(SRS['G1_Points'], p_a.coefficients())])
b_s = sum([t[0]*t[1] for t in zip(SRS['G1_Points'], p_b.coefficients())])
c_s = sum([t[0]*t[1] for t in zip(SRS['G1_Points'], p_c.coefficients())])
a_s,b_s, c_s

((91 : 66 : 1), (26 : 45 : 1), (91 : 35 : 1))

In [577]:
def commit(polynomial):
        return sum([t[0]*t[1] for t in zip(SRS['G1_Points'], polynomial.coefficients(sparse= False))])

We can also check ourselves and previos formula by evaluating the polynomials at "secret" $s$ point and multiply in by $G_1$

In [380]:
p_a(s)*G1, p_b(s)*G1, p_c(s)*G1

((91 : 66 : 1), (26 : 45 : 1), (91 : 35 : 1))


### Round 2

Now we as the prover have to create a polynomial that is absorbs all the information about the copy constraints, and then we will commit to it evaluations as it was done previously:

Now we generate three more random values 


In [257]:
bs = [FF(i) for i in [7,4,11,12,16,2,14,11,7]]

In [258]:
# z = R([b[8],b[7],b[6]])*Z_H


In [285]:
def numerator(i):
    m1 = a[i-1] + betta*w^(i-1) + gamma
    m2 = b[i-1] + betta* k1*w^(i-1) + gamma
    m3 = c[i-1] + betta * k2*w^(i-1) + gamma
    return m1*m2*m3
    

In [286]:
def denominator(i):
    m1 = a[i-1] + betta*s_sigma_1(w^(i-1)) + gamma
    m2 = b[i-1] + betta*s_sigma_2(w^(i-1)) + gamma
    m3 = c[i-1] + betta*s_sigma_3(w^(i-1)) + gamma
    return m1*m2*m3


In [272]:
def acc(i):
    if i==0:
        return 1
    else:
        return acc(i-1)*numerator(i)/denominator(i)

In [360]:
betta = 12
gamma = 13
print('acc evaluations at domain:',[acc(i) for i in range(4)])
acc_s = R.lagrange_polynomial([t for t in zip ( domain[0], [acc(i) for i in range(4)])])

acc evaluations at domain: [1, 3, 9, 4]


In [363]:
z = R([bs[8],bs[7],bs[6]])*Z_H + acc_s
print(z)

14*x^6 + 11*x^5 + 7*x^4 + 14*x^3 + 8*x^2 + 5*x + 10


In [372]:
commit(z)

(32 : 59 : 1)

### Round 3

In [429]:
PI = R(0)
alpha = 15
L1 = R.lagrange_polynomial(zip(domain[0], [1,0,0,0]))
L1

13*x^3 + 13*x^2 + 13*x + 13

In [423]:
zw = R([z.coefficients()[i]*w^i for i in range(len(z.coefficients()))])
zw

3*x^6 + 10*x^5 + 7*x^4 + 12*x^3 + 9*x^2 + 3*x + 10

In [487]:
t1zh = (p_a*p_b*f_q_M + p_a *f_q_L + p_b*f_q_R + p_c * f_q_O + PI + f_q_const)
t1zh

9*x^13 + 7*x^12 + 8*x^11 + 9*x^10 + 5*x^9 + 10*x^8 + 5*x^7 + 3*x^6 + 3*x^5 + 16*x^4 + 4*x^3 + 5*x^2 + 1

In [481]:
t21 = alpha*(p_a + R([0,betta]) + gamma)
t21

3*x^5 + 9*x^4 + 11*x^3 + 11*x^2 + 15*x + 14

In [482]:
t22 = (p_b + R([0,betta*k1]) + gamma)
t22

11*x^5 + 12*x^4 + 13*x^3 + 14*x^2 + 16*x + 8

In [484]:
t23 = p_c + R([0,betta*k2]) + gamma
t23

16*x^5 + 2*x^4 + 4*x^3 + 11*x^2 + 8*x

In [489]:
t2zh = t21*t22*t23*z
t2zh

14*x^21 + 14*x^20 + 2*x^19 + 15*x^18 + 8*x^17 + 16*x^16 + 16*x^15 + 4*x^14 + 12*x^13 + x^12 + 12*x^11 + 12*x^10 + 7*x^9 + 8*x^8 + 13*x^7 + 10*x^6 + 3*x^4 + 12*x^3 + x^2 + x

In [501]:
t31 = alpha*(p_a+betta*s_sigma_1 + gamma)
print(t31)
t32 = (p_b + betta * s_sigma_2 + gamma)
print( t32)
t33 = (p_c + betta*s_sigma_3+gamma)
print(t33)
print (zw)
t3zh = -t31 * t32* t33*zw
print(t3zh)

3*x^5 + 9*x^4 + 3*x^3 + 9*x^2 + 16*x + 16
11*x^5 + 12*x^4 + 8*x^3 + 9*x + 5
16*x^5 + 2*x^4 + 2*x^3 + 13*x^2 + 5*x + 4
3*x^6 + 10*x^5 + 7*x^4 + 12*x^3 + 9*x^2 + 3*x + 10
14*x^21 + 10*x^20 + x^18 + 4*x^17 + 9*x^16 + 7*x^14 + 9*x^13 + 12*x^12 + 4*x^11 + 16*x^10 + 2*x^9 + 7*x^8 + 2*x^7 + 9*x^6 + 11*x^5 + 9*x^4 + 7*x^3 + 10*x^2 + 3*x + 13


In [509]:
t4zh = (z-1)*L1 * alpha^2
t4zh

14*x^9 + 8*x^8 + 15*x^7 + 12*x^6 + 6*x^5 + 2*x^3 + 5*x^2 + 14*x + 9

In [533]:
t = R((t1zh+t2zh+t3zh+t4zh)/Z_H)
t.coefficients(sparse=False)

[11, 16, 13, 9, 0, 13, 13, 8, 1, 2, 10, 1, 15, 6, 16, 2, 7, 11]

In [554]:
t_lo=R(t.coefficients(sparse=False)[:6])
t_mid = R(t.coefficients(sparse=False)[6:12])
t_hi = R(t.coefficients(sparse=False)[12:])
print(t_lo)
print(R(t_lo))
print(R(t_mid))
print(R(t_hi))

13*x^5 + 9*x^3 + 13*x^2 + 16*x + 11
13*x^5 + 9*x^3 + 13*x^2 + 16*x + 11
x^5 + 10*x^4 + 2*x^3 + x^2 + 8*x + 13
11*x^5 + 7*x^4 + 2*x^3 + 16*x^2 + 6*x + 15


In [567]:
t_check = t_lo + t_mid * R (x^6)  + t_hi*R(x^(2*6))
t == t_check

True

In [580]:
commit(t_lo),commit(t_mid),commit(t_hi)

((12 : 32 : 1), (26 : 45 : 1), (91 : 66 : 1))

### Round 4

In [581]:
zeta = 5

In [588]:
a_zeta = comm_a(zeta)
b_zeta = comm_b(zeta)
c_zeta = comm_c(zeta)
s_sigma_1_zeta = s_sigma_1(zeta)
s_sigma_2_zeta = s_sigma_2(zeta)
s_sigma_3_zeta = s_sigma_3(zeta)
t_zeta = t(zeta)
zw_zeta = zw(zeta)
print (a_zeta, b_zeta, c_zeta, s_sigma_1_zeta, s_sigma_2_zeta, s_sigma_3_zeta, t_zeta, zw_zeta)

15 13 5 1 12 13 1 15


In [616]:
r1 = a_zeta * b_zeta * f_q_M + \
    a_zeta * f_q_L + b_zeta * f_q_R + \
    c_zeta * f_q_O + PI(zeta) + f_q_const
r21 = (a_zeta + betta * zeta + gamma) * \
    (b_zeta + betta*k1*zeta + gamma) * \
    (c_zeta + betta * k2*zeta + gamma) * z*alpha
r22 = (a_zeta + betta* s_sigma_1_zeta + gamma) *\
    (b_zeta + betta * s_sigma_2_zeta + gamma) * \
    (c_zeta + betta * s_sigma_3 + gamma)*zw*alpha
r2 = r21 -  r22
r3 = alpha^2*(z-1)*L1(zeta)
r4 = Z_H(zeta)* (t_lo + zeta^6 * t_mid + zeta^12 * t_hi)
r = r1 - r2 +r3 - r4
r



6*x^5 + 2*x^4 + 16*x^2 + 11*x + 14

In [608]:
a_zeta * b_zeta * f_q_M

8*x^3 + 2*x^2 + 9*x + 6

In [610]:
a_zeta * f_q_L

2*x^3 + 9*x^2 + 15*x + 8

In [612]:
b_zeta * f_q_R

4*x^3 + x^2 + 13*x + 16

In [617]:
c_zeta * f_q_O,PI(zeta),f_q_const

(12, 0, 0)

In [622]:
r21,r22,r2

(8*x^6 + 16*x^5 + 4*x^4 + 8*x^3 + 7*x^2 + 15*x + 13,
 0,
 8*x^6 + 16*x^5 + 4*x^4 + 8*x^3 + 7*x^2 + 15*x + 13)

In [623]:
r3

8*x^6 + 16*x^5 + 4*x^4 + 8*x^3 + 7*x^2 + 15*x + 10